# Laboratorio 6c

Classificazione basic/advanced. Si richiedere l'uso (o meno) del dataset su basicness per fare classificazione automatica (binaria, basic/advanced) su nuovi termini e/o synset presi in esame.

## Import delle librerie

In [1]:
base_folder = './data'
!pip install -U sentence-transformers
import nltk
nltk.download('wordnet')
import pandas as pd
import json
from nltk.corpus import wordnet as wn
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
import numpy as np
import gensim.downloader
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.neighbors import NearestCentroid
from sklearn import tree

[nltk_data] Downloading package wordnet to /Users/mario/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Data loading and pre-processing

I dati sono trasformati dal formato *json* ad un DataFrame di *pandas*, il synset annotato è convertito nell'effettivo oggetto Synset di *nltk*.

In [2]:
with open(f'{base_folder}/dataset_basic_advanced_TLN2023/1.json') as f:
    dataset = json.load(f)
del dataset['i']
del dataset['date']

dataset = pd.DataFrame(dataset)
dataset['answers'] = dataset['answers'].map(lambda x: 1 if x == 'advanced' else 0)
dataset['synset'] = dataset['dataset'].map(lambda x: wn.synset(x.split(':')[0].split('(')[1][1:-2]))
dataset['words'] = dataset['dataset'].map(lambda x: list(w.strip() for w in x.split(':')[1].split('|')[0].split(',')))
dataset['def'] = dataset['dataset'].map(lambda x: x.split(':')[2].strip())
dataset['isHard'] = dataset['isHard'].map(lambda x: 1 if x == True else 0)
del dataset['dataset']

### Caricamento dei dati per ottenere il valore medio

Una volta caricato il primo dataset si procede a caricare gli altri file disponibili, ogni file corrisponde ad una annotazione diversa.

Per le colonne relative alle risposte, alla difficoltà dell'annotazione e al tempo impiegato viene salvato il valore medio tra tutti quelli annotati.

Inoltre, per ricondurre i dati al problema della classificazione la colonna 'answer' viene confrontata con un valore di threshold. Se i diversi annotatori sono mediamente d'accordo sulla difficoltà del termine si salva il valore 1, ovvero quello relativo ad un termine *advanced*.

In [3]:
for i in range(2, 11):
    with open(f'{base_folder}/dataset_basic_advanced_TLN2023/{i}.json') as f:
        for idx, elem in enumerate(json.load(f)['answers']):
            dataset.iat[idx, 1] += 1 if elem == 'advanced' else 0
    with open(f'{base_folder}/dataset_basic_advanced_TLN2023/{i}.json') as f:
        for idx, elem in enumerate(json.load(f)['isHard']):
            dataset.iat[idx, 0] += 1 if elem == True else 0
    with open(f'{base_folder}/dataset_basic_advanced_TLN2023/{i}.json') as f:
        for idx, elem in enumerate(json.load(f)['timeDiffs']):
            dataset.iat[idx, 2] += elem
dataset['answers'] = dataset['answers'].map(lambda x: 1 if x/10 > 0.6 else 0)
dataset['isHard'] = dataset['isHard'].map(lambda x: x/10)
dataset['timeDiffs'] = dataset['timeDiffs'].map(lambda x: x/10)

display(dataset)

,isHard,answers,timeDiffs,synset,words,def
0,0.0,0,3.3025,Synset('war.n.01'),"[war, warfare]",the waging of armed conflict against an enemy
1,0.0,1,2.8927,Synset('fiefdom.n.01'),[fiefdom],the domain controlled by a feudal lord
2,0.0,0,2.5290,Synset('bed.n.03'),"[bed, bottom]",a depression forming the ground under a body o...
3,0.0,1,3.6712,Synset('return_on_invested_capital.n.01'),"[return on invested capital, return on investm...","(corporate finance) the amount, expressed as a..."
4,0.1,0,4.0939,Synset('texture.n.02'),[texture],the essential quality of something
...,...,...,...,...,...,...
499,0.1,0,1.4891,Synset('reading.n.03'),"[reading, meter reading, indication]",a datum about some physical state that is pres...
500,0.0,1,2.2836,Synset('sanctimoniousness.n.01'),"[sanctimoniousness, sanctimony]",the quality of being hypocritically devout
501,0.0,1,1.6128,Synset('chalcedony.n.01'),"[chalcedony, calcedony]",a milky or greyish translucent to transparent ...
502,0.0,1,1.4985,Synset('stopcock.n.01'),"[stopcock, cock, turncock]",faucet consisting of a rotating device for reg...


## Uso dei vettori GloVe

Per fare la classificazione si possono utilizzare i vettori di GloVe. Il modello pre-trained viene scaricato tramite la libreria *gensim* e si effettua una ricerca tramite il metodo *search* del termine nell'elenco di vettori. Se non c'è nessun vettore disponibile si restituisce un valore fittizio pari a 0.

In [4]:
def search(vectors, words):
    for word in words:
        if len(word.split(' ')) > 1:
            tmp = []
            for sub_word in word.split(' '):
                if sub_word in vectors:
                    tmp.append(vectors[sub_word])
            if len(tmp) > 0:
                return sum(tmp) / len(tmp)
        if word in vectors:
            return vectors[word]
    return [0]*len(vectors['test'])

In [7]:
glove_models = ['glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-300']

In [8]:
for model in glove_models:
    glove = gensim.downloader.load(model)
    dataset[model] = dataset['words'].map(lambda x: search(glove, x))

## Vettori di Sentence-Bert

Un altro possibile approccio è usare i vettori basandosi sulle definizioni dei synset. Si sfrutta quindi il modello Sentence-Bert per ottenere la rappresentazione vettoriale.

In [9]:
bert = SentenceTransformer('all-mpnet-base-v2')
dataset['sentence_bert'] = dataset['def'].map(lambda x: bert.encode(x))

## Estrazione di feature a mano

Alcune feature possono essere estratte manualmente a partire dal dataset. I vettori così ottenuti saranno di dimensioni sensibilmente inferiori e catturano caratteristiche che hanno un significato anche se ispezionata da un occhio umano.

La prima versione di tali vettori sfrutta tutte le informazioni ricavabili dal dataset, ovvero:
- Difficoltà percepita dagli annotatori nel classificare il senso
- Tempo impiegato per la classificazione
- Lunghezza del termine
- Frequenza del termine (secondo quanto annotato in WordNet)
- PoS del termine (convertita in numero)

In [10]:
manual_embeddings = []
pos_num = []
for idx in range(len(dataset)):
  vec = [dataset['isHard'][idx], 
         dataset['timeDiffs'][idx], 
         len(dataset['synset'][idx].lemmas()[0].name()),
         dataset['synset'][idx].lemmas()[0].count()]
  pos = dataset['synset'][idx].pos()
  if pos not in pos_num:
    pos_num.append(pos)
  vec.append(pos_num.index(pos))
  manual_embeddings.append(vec)
dataset['manual_vectors'] = manual_embeddings

La variante riportata di seguito cerca di generalizzare quanto visto sopra, in modo che sia applicabile non solo ai dati disponibili ma ad ogni senso di WordNet, ed è per questo motivo che non si utilizzano le annotazioni 'isHard' e 'timeDiffs'.

In [11]:
manual_embeddings = []
pos_num = []
for idx in range(len(dataset)):
  vec = [len(dataset['synset'][idx].lemmas()[0].name()),
         dataset['synset'][idx].lemmas()[0].count()]
  pos = dataset['synset'][idx].pos()
  if pos not in pos_num:
    pos_num.append(pos)
  vec.append(pos_num.index(pos))
  manual_embeddings.append(vec)
dataset['manual_vectors_general'] = manual_embeddings

In [12]:
display(dataset)

,isHard,answers,timeDiffs,synset,words,def,glove-wiki-gigaword-50,glove-wiki-gigaword-100,glove-twitter-25,glove-twitter-50,glove-twitter-100,glove-wiki-gigaword-300,sentence_bert,manual_vectors,manual_vectors_general
0,0.0,0,3.3025,Synset('war.n.01'),"[war, warfare]",the waging of armed conflict against an enemy,"[0.36544, -0.15746, -0.23966, -1.0307, -0.0706...","[-0.39505, 1.0285, -0.21556, 0.36596, -0.34455...","[1.0727, -0.69577, -0.83886, -0.71605, 0.26884...","[0.53829, 0.64019, -0.093889, -0.3971, 0.08496...","[-0.12546, 0.73857, -0.86254, -0.031499, 0.095...","[-0.33042, 0.22503, 0.46759, -0.28312, 0.24944...","[0.007489369, -0.04419364, 0.048122793, -0.024...","[0.0, 3.3024999999999998, 3, 78, 0]","[3, 78, 0]"
1,0.0,1,2.8927,Synset('fiefdom.n.01'),[fiefdom],the domain controlled by a feudal lord,"[0.66869, -0.3902, -0.27852, 0.11272, -0.3041,...","[-0.05284, -0.61082, 0.94874, -0.0053948, 0.44...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.46706, -0.16593, 0.61854, 0.41386, 0.43572,...","[0.05618644, 0.0498442, -0.02059003, -0.008219...","[0.0, 2.8927000000000005, 7, 0, 0]","[7, 0, 0]"
2,0.0,0,2.5290,Synset('bed.n.03'),"[bed, bottom]",a depression forming the ground under a body o...,"[0.75344, 0.96383, 0.10188, -0.67978, 0.38461,...","[-0.83528, 0.57023, 0.19219, -0.025946, -0.500...","[-1.484, 0.11743, 0.72771, 0.048489, -0.16798,...","[-0.2065, -0.36727, 0.38399, -0.042979, -1.135...","[0.07052, -0.099901, 0.94344, 0.028017, -0.360...","[-0.20441, -0.082417, -0.056366, -0.17798, -0....","[-0.029942425, -0.06869351, -0.00899393, -0.03...","[0.0, 2.529, 3, 2, 0]","[3, 2, 0]"
3,0.0,1,3.6712,Synset('return_on_invested_capital.n.01'),"[return on invested capital, return on investm...","(corporate finance) the amount, expressed as a...","[0.69420004, 0.11292124, 0.26995924, 0.0424982...","[-0.024894997, -0.08278751, 0.21433249, 0.1391...","[-0.109346256, 0.29860875, -0.4278425, -0.2325...","[-0.14733, 0.014635012, 0.01589248, 0.02117500...","[0.3407675, 0.19607, 0.0130324885, -0.22935, -...","[0.073894635, -0.051997, -0.052279994, -0.1466...","[-0.052810527, -0.07921334, 0.0026665344, -0.0...","[0.0, 3.6712000000000002, 26, 0, 0]","[26, 0, 0]"
4,0.1,0,4.0939,Synset('texture.n.02'),[texture],the essential quality of something,"[0.046497, -0.19038, -1.2709, 0.36144, 1.0203,...","[-0.74544, 0.45168, 0.27644, 0.078507, -0.1495...","[-0.040664, -1.1883, -0.15649, 0.30475, 0.9954...","[-0.65289, -0.95866, -0.5199, 0.65781, 1.0537,...","[-0.51789, -0.97407, -0.3964, 0.67164, 0.80225...","[-0.13618, 0.10435, -0.098815, -0.36159, -0.32...","[0.008202355, 0.035106614, -1.293693e-05, -0.0...","[0.1, 4.0939, 7, 3, 0]","[7, 3, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,0.1,0,1.4891,Synset('reading.n.03'),"[reading, meter reading, indication]",a datum about some physical state that is pres...,"[-0.86467, 0.52566, -0.23499, -1.4381, 0.62089...","[0.22275, 0.70474, 0.018312, 0.064222, 0.07811...","[0.20497, 0.68135, 0.90055, 0.10242, 0.25843, ...","[0.79712, 0.38046, 0.68029, -0.1799, 0.1954, 0...","[-0.059789, 0.076035, -0.0072208, -0.044774, -...","[-0.37081, -0.34448, 0.1233, -0.43801, -0.0213...","[-0.080179796, -0.059805144, 0.0045927037, -0....","[0.1, 1.4891, 7, 3, 0]","[7, 3, 0]"
500,0.0,1,2.2836,Synset('sanctimoniousness.n.01'),"[sanctimoniousness, sanctimony]",the quality of being hypocritically devout,"[0.19583, -0.30178, -0.58289, -0.56737, -0.190...","[0.23115, -0.15386, 0.70575, -0.1388, -0.29811...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.35571, -0.018719, 0.24613, 0.46533, -0.2615...","[-0.016181672, 0.058728777, 0.017140668, 0.026...","[0.0, 2.2836000000000003, 17, 0, 0]","[17, 0, 0]"
501,0.0,1,1.6128,Synset('chalcedony.n.01'),"[chalcedony, calcedony]",a mil

## Test della classificazione

Per testare le rappresentazioni ottenute si esegue il task di classificazione tramite i modelli di *Support Vector Machine*, *Nearest Centroid* e *Decision Tree*.

Vengono quindi stampati i report che rappresentano le performance ottenute da ogni classificatore usando i vari embedding disponibili.

In [13]:
embeddings_columns = ['glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-300',
                      'sentence_bert', 'manual_vectors', 'manual_vectors_general']

In [14]:
def eval_classification(classifier, col, name):
  embeddings = []
  for dim in np.array(dataset[col]):
    vec = [x for x in dim]
    embeddings.append(vec)

  x_train, x_test, y_train, y_test = train_test_split(embeddings, np.array(dataset['answers']), test_size=.1)
  cls = classifier()
  cls.fit(x_train, y_train)
  y_pred = cls.predict(x_test)
  print(name, col)
  print(classification_report(y_test, y_pred, target_names=['Basic', 'Advanced']))
  print()

## Risultati

Scorrendo i vari report ottenuti, si può notare come alcune accoppiate classificatore-vettore funzionino meglio di altre.

Le combinazioni di classificatore ed embedding migliori raggiungono punteggi di f1-score pari a 0.90.

Anche i vettori creati manualmente raggiungono risultati ragionevoli. Questo potrebbe significare che c'è una forte correlazione tra le feature considerate e la *basicness* di un termine.

Con i vari metodi esposti, fatta eccezione per i vettori che utilizzano 'isHard' e 'timeDiffs', è possibile annotare la *basicness* d'intere risorse semantiche come WordNet o, ancora più in generale, della maggior parte delle parole della lingua inglese in quanto sono contenute nei modelli *GloVe*.

In [15]:
for col in embeddings_columns:
  for classifier, name in [(svm.SVC, 'SVM'), (NearestCentroid, 'KNC'), (tree.DecisionTreeClassifier, 'DecisionTree')]:
    eval_classification(classifier, col, name)

SVM glove-wiki-gigaword-50
              precision    recall  f1-score   support

       Basic       0.70      0.88      0.78        26
    Advanced       0.83      0.60      0.70        25

    accuracy                           0.75        51
   macro avg       0.77      0.74      0.74        51
weighted avg       0.76      0.75      0.74        51


KNC glove-wiki-gigaword-50
              precision    recall  f1-score   support

       Basic       0.96      0.93      0.95        28
    Advanced       0.92      0.96      0.94        23

    accuracy                           0.94        51
   macro avg       0.94      0.94      0.94        51
weighted avg       0.94      0.94      0.94        51


DecisionTree glove-wiki-gigaword-50
              precision    recall  f1-score   support

       Basic       0.66      0.76      0.70        25
    Advanced       0.73      0.62      0.67        26

    accuracy                           0.69        51
   macro avg       0.69      0.69   